# Introduction aux réseaux de neurones

## Partie 1 - Classification

In [ ]:
from __future__ import print_function

%load_ext autoreload
%autoreload 2

import numpy as np
import math
import random
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

from dojo.dataset import Cifar10Dataset

### Chargement du dataset

Durant ce dojo, nous utiliserons le dataset CIFAR10. Ce dataset comprend 60000 images RGB 32 par 32 de 10 classes différentes:

0. airplane
1. automobile
2. bird
3. cat
4. deer
5. dog
6. frog
7. horse
8. ship
9. truck

Il existe également le dataset CIFAR100, qui a lui 100 classes distinctes. Pour plus de détails, voir le [site officiel](https://www.cs.toronto.edu/~kriz/cifar.html).

Évaluez les cellules suivante pour télécharger le jeu de données. Normalement, vous devriez voir qu'il y a 50000 images dans les données d'entraînement et 10000 images dans les données de test.

In [ ]:
cifar10 = Cifar10Dataset()

In [ ]:
print(cifar10.x_train.shape)
print(cifar10.y_train.shape)
print(cifar10.x_test.shape)
print(cifar10.y_test.shape)

In [ ]:
# Visualize some examples from the dataset.
# We show a few examples of training images from each class.
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = len(classes)
samples_per_class = 7
for y, cls in enumerate(classes):
    idxs = np.flatnonzero(cifar10.y_train == y)
    idxs = np.random.choice(idxs, samples_per_class, replace=False)
    for i, idx in enumerate(idxs):
        plt_idx = i * num_classes + y + 1
        plt.subplot(samples_per_class, num_classes, plt_idx)
        plt.imshow(cifar10.x_train[idx].astype('uint8'))
        plt.axis('off')
        if i == 0:
            plt.title(cls)
plt.show()

### Le problème de classification

Regardons la première image:

In [ ]:
print(cifar10.x_train[0])
print(cifar10.y_train[0])
plt.imshow(cifar10.x_train[0].astype('uint8'))

Cette image est comprise par un humain comme étant une grenouille. Par contre, pour un ordinateur, cette image n'est rien de plus q'un tableau de 32 x 32 x 3 nombres à virgule flottante. Le problème de classification d'image est donc le suivant:
 
 
> Étant donné des images $x_i$ et leurs étiquettes $y_i$, trouvez la fonction $f(X): x \rightarrow y$ qui maximise le bon nombre d'associations entre les images et les étiquettes et qui se généralise à des images inconnues.


### k-Nearest Neighbor

Un des algorithmes les plus simple pour résoudre le problème de classification est k-Nearest Neighbor (kNN). k est un hyper-paramètre de l'algorithme, c'est-à-dire un paramètre qui est choisi par le programmeur de l'algorithme en fonction de l'application. L'algorithme fonctionne comme suit:

Étant donné un point de test $x$, on regarde les $k$ voisins les plus proches et on procède à un vote de majorité sur la classe que devrait avoir $x$. Voici un exemple visuel où les données ont deux dimensions (x, y) et il y a trois classes possibles (bleu, rouge et vert).

![kNN](http://cs231n.github.io/assets/knn.jpeg)

Pour obtenir les voisins les plus proches, il faut d'abord se choisir une métrique de distance. Souvent, on utilise la distance L2 (ou distance euclidienne). Dans le cas 2D, on a:

$$
d = \sqrt{x^2 + y^2}
$$

On peut utiliser la même métrique pour des images en comparant pixel par pixel chacune des valeurs de r, g et b.

#### Exercice

Complétez le fichier k_nearest_neighbor.py situé dans le dossier dojo. Dans un premier temps, complétez la fonction `compute_distances_two_loops`. Vous pouvez viusaliser le résultat avec le code suivant:

> **NOTE** Nous utiliserons ici que le dixième du dataset. Les images rgb sont redimensionnées en vecteur.

In [ ]:
from dojo.k_nearest_neighbor import KNearestNeighbor

n_of_train = 5000
n_of_test = 500

x_train, y_train = cifar10.x_train[:n_of_train], cifar10.y_train[:n_of_train]
x_test, y_test = cifar10.x_test[:n_of_test], cifar10.y_test[:n_of_test]
x_train = x_train.reshape((n_of_train, -1))
x_test = x_test.reshape((n_of_test, -1))

print(x_train.shape)

classifier = KNearestNeighbor()
classifier.train(x_train, y_train)

dists = classifier.compute_distances_two_loops(x_test) # May take some time to compute
print(dists.shape)

In [ ]:
plt.imshow(dists, interpolation='none')
plt.show()

- Que représentent les colonnes claires? Les colonnes foncées?

Maintenant, implémentez la fonction `predict_labels`.

In [ ]:
# We use k = 1 (which is Nearest Neighbor).
y_test_pred = classifier.predict_labels(dists, k=3)

# Compute and print the fraction of correctly predicted examples
num_correct = np.sum(y_test_pred == y_test)
accuracy = float(num_correct) / n_of_test
print('Got %d / %d correct => accuracy: %f' % (num_correct, n_of_test, accuracy))

Votre algorithme devrait obtenir environ 27% d'accuracy. Vous pouvez jouer avec le paramètre $k$ pour regarder si les résultats s'améliorent ou non. Quelle est la meilleure valeur de k?

#### Discussion
- Peut-on utiliser kNN dans la forme présentée plus haut pour la classification de grands dataset?
- Nommer une limitation de l'algorithme.
- Comment avez-vous choisi la valeur de k? Avez-vous utilisé les données de test?

## Partie 2 - Backpropagation

Dans la partie précédente, nous avons vu une manière assez limitée de faire de la classification. En effet, il faut précalculer une matrice de distances, puis itérer sur cette matrice pour faire un vote de majorité, ce qui est extrèmement coûteux. Idéalement, nous aimerions avoir un algorithme qui apprend itérativement en s'améliorant à chaque étape. L'algorithme aurait la forme suivante:

1. On donne à l'algorithme une donnée ou une mini-batch de données (ex. 64 points) au hasard.
2. L'algorithme utilise ses paramètre actuels pour prédire les étiquettes des données.
3. L'algorithme évalue l'erreur produite sur cette évaluation
4. L'algorithme évalue le rôle de chaque paramètre sur cette erreur.
5. L'algorithme ajuste ces paramètres.
6. goto 1.

Les étapes 3. et 4. sont le nerf de la guerre. Ce qui est décrit en 3. est s'appelle la *loss function*. L'étape 4. est résolue par une technique qui s'appelle la *backpropagation*. 

Le but est de trouver un minimum local où la fonction de loss est la plus petite possible en se déplacant dans le sens du gradient.

![gradient-descent](http://sebastianruder.com/content/images/2016/09/saddle_point_evaluation_optimizers.gif)


Voyons ces étapes en détail à l'aide d'un exemple jouet avant de s'attaquer au problème de classification.

### Fitting de courbe polynomiale

Le dataset suivant est constitué de points $(x, y)$ où $y = ax^2 + bx + c + \sigma$, où sigma représente du bruit dans la fonction.

> **ATTENTION** En temps normal, on ne connais pas la distribution statistique des données d'entraînement, ce qui rend le problème extrêmement plus complexe.

Notre algorithme tenteras d'estimer les paramètre a, b et c.
Dans un premier temps, initialisons le dataset et les paramètres de la fonction à apprendre:



In [ ]:
from dojo.dataset import PolynomialDataset

min_param, max_param = (0, 100)
poly_data = PolynomialDataset(min_param=min_param, max_param=max_param)
n_of_train = poly_data.x_train.shape[0]
n_of_test = poly_data.x_test.shape[0]
print(n_of_train, n_of_test)


### *Loss function*
Écrivez la *loss function*. Il s'agit de la différence entre le vrai $y$ et le $y$ prédit.

In [ ]:
def loss_function(y_predicted, y_ground_truth):
    return y_ground_truth - y_predicted

### Backprop

Maintenant, il faut regarder comment a, b et c affectent l'erreur sur la prédiction. Pour se faire, il faut calculer les dérivées partielles de la fonction. Cela fait du sens puisque ces dérivées partiellent indiquent comment chaque paramètre influence la sortie de la fonction.

$$
f(x) = ax^2 + bx + c
$$

$$
\frac{\partial f}{\partial a} = x^2
$$

$$
\frac{\partial f}{\partial b} = x
$$

$$
\frac{\partial f}{\partial c} = 1
$$

#### Exercice
Écrivez la fonction qui retourne le gradient (vecteur des dérivées partielles). Ce vecteur de gradient doit être modulé (multiplié) par le loss.

In [ ]:
def gradient(x, loss):
    return np.array([x**2, x, 1]) * loss

### Entraînement

In [ ]:
learning_rate = 1e-2

a, b, c = 0.0, 0.0, 0.0

print(a, b, c)

for i in range(3000):
    random_index = random.randint(0, n_of_train - 1)
    x = poly_data.x_train[random_index]
    y = poly_data.y_train[random_index]
    y_predicted = a * x**2 + b * x + c
    loss = loss_function(y_predicted, y)
    da, db, dc = gradient(x, loss)
    
    a += learning_rate * da
    b += learning_rate * db
    c += learning_rate * dc
    
    if i % 20 == 0:
        print("Step %d: a: %f b: %f c: %f" % (i, round(a, 2), round(b, 2), round(c, 2)))
        print("---loss: %f" % loss)
        
print("TRAINING FINISHED")
print("ESTIMATED PARAMS a: %f b: %f c: %f" % (round(a, 2), round(b, 2), round(c, 2)))
print("ACTUAL PARAMS a: %d b: %d c: %d" % (poly_data.a, poly_data.b, poly_data.c))

#### Discussion:
- Comment se comporte le loss au fil du temps?
- Décrivez le comportement des paramètres a, b, c au fil du temps.
- À quoi sert le learning rate? Modifiez la valeur et observez le comportement.
- Modifier les valeurs initiales des paramètres et observez le comportement.
- Comment peut-on vérifier que les paramètres a, b et c sont les bons?

## Partie 3 - Classification linéaire

Dans cette partie, nous développerons un modèle de classification linéaire pour le dataset CIFAR10. Le but est d'avoir une approche itérative de descente de gradient comme à la partie 2. Si vous comprenez bien ce modèle, vous comprendrez aussi les réseaux de neurones, qui sont simplement la combinaison de plusieurs modèles linéaires.


Revenons au problème de classification. Rappel:

> Étant donné des images $x_i$ et leurs étiquettes $y_i$, trouvez la fonction $f(X): x \rightarrow y$ qui maximise le bon nombre d'associations entre les images et les étiquettes et qui se généralise à des images inconnues.


On peut décomposer la fonction $f$ en plusieurs fonctions. Au lieu d'avoir une fonction unique qui retourne directment l'étiquette $y$ de l'image, on peut avoir une fonction par classe du dataset et chaque fonction retourne le score associé à l'image en entrée qui indique à quel point il est probable que l'image est cette classe. Par exemple, si le dataset a seulement deux classes, 0 pour chien et 1 pour chat, nous aurions deux fonctions $f_0(X)$ et $f_1(X)$. Pour une image donnée $x_i$, si on a $f_0(x_i) = 0.6$ et $f_1(x_i) = 0.2$, on en déduit que l'image $x_i$  est un chien, car selon les fonctions, il est plus probable que l'image soit un chien. Voici une représentation visuelle des fonction $f_j(x)$. À noter, la figure est en 2D, mais dans le cas réel, les fonctions séparent les images dans un espace à $32 \times 32 \times 3 = 3072$ dimensions.

![classification figure](http://cs231n.github.io/assets/pixelspace.jpeg)

La classification linéaire est un modèle très simple pour représenter les fonctions $f_j(x)$. La formulation est la suivante:

$$
f_j(x) = w_j \cdot x + b_j
$$


Dans la formule précédente, le vecteur $w_j$ représente les poids (ou *weights* en anglais) à associer à chaque pixel de l'image $x$ et $b_j$ est un scalaire qui représente le biais, c'est-à-dire si à priori il est plus probable d'observer la classe $j$.


Pour que le calcul soit plus efficace, il est possible de calculer tous les scores $f_j(x)$ en même temps. Il suffit de mettre tous les vecteurs $w_j$ dans une matrice $W$ où chaque ligne $j$ est $w_j$ et de faire le produit matriciel $Wx$, puis additionner un vecteur $b$ qui contient les biais. Cela revient exactement au même que de faire les produit scalaires séparemment. Voici une représentation visuelle cette opération:


> **NOTE** Pour simplifier les calculs, $x$ est redimensionné en un vecteur de 3072 éléments. Ainsi, il est très facile d'associer un point à chaque pixel et multiplier ces points à $x$, puisque cela se résume en un produit scalaire.

> **NOTE** Pour simplifier encore plus les calculs, on peut ajouter un élément 1 dans $x$ et une colonne à $W$. Cela revient exactement au même que d'additionner séparemment $b$.

![weight matrix](http://cs231n.github.io/assets/imagemap.jpg)


### Généralisation

La généralisation est un élément primordial en machine learning. C'est bien beau d'être capable de classifier correctement les données d'entraînement, mais il faut à tout prix éviter le phénomène d'*overfitting*. L'exemple classique est de trouver la courbe qui passe par une série de points. Si on a $n$ points, il est possible de trouver un polynôme de degré $n + 1$ qui passe parfaitement par tous les points. Par contre, cette courbe risque de moins bien performer qu'un modèle plus simple. pour passer à travers les autres points de la distribution que l'on n'as pas encore observé.

![overfitting](https://upload.wikimedia.org/wikipedia/commons/thumb/6/68/Overfitted_Data.png/300px-Overfitted_Data.png)


Généralement, en machine learning, on sépare les données en deux groupes: *training* et *testing*. Ainsi, on peut évaluer la performance du modèle sur les données inconnues en utilisant des données qui n'ont pas servi à l'entraînement.

### *Loss function* - Softmax

Contrairement à la partie 2 où nous voulions faire de la *régression*, on ne peut pas directement calculé l'erreur sur l'estimé de la fonction. Pour la classification, nous untiliserons une *loss function* appelée **softmax**. 

Disons qu'on veut classifier un dataset qui a uniquement deux classes: chien et chat. Puisqu'il y a deux classes, la fonction retourne un vecteur de deux éléments. Si, étant donné une image de chien, la fonction retourne le vecteur suivant:

$$
f_j(x) = [-0.04, 0.12]
$$

On peut interpréter ces valeurs **comme le log de la probabilité (non normalisée)** que l'image appartienne à chaque classe. Ainsi, pour les classes 0 (chien) et 1 (chat), on a:

$$
P(0\,|\,x_i,W,b) = \frac{e^{-0.04}}{e^{-0.04} + e^{0.12}} = 0.46
$$

$$
P(1\,|\,x_i,W,b) = \frac{e^{0.12}}{e^{-0.04} + e^{0.12}} = 0.54
$$

La fonction estime donc qu'il est plus probable que l'image soit de classe 1 (chat), puisque la probabilité est de 54% pour cette classe. On remarque également que les probabilités sont maintenant normalisées (ont une somme de 1).

La formule générale est la suivant:

$$
P(y_i\,|\,x_i,W,b) = \frac{e^{f_{y_i} + \log C}}{\sum_j e^{f_{y_i} + \log C}}
$$

où la constante $\log C = -\max_jf_j$ est ajoutée uniquement pour la stabilité numérique.


Maintenant qu'on peut mettre une valeur sur la justesse de la prédiction de la classification, on peut facilement calculer le *loss*. Reprenons l'exemple précédent. On sait que l'image est une image de chien, avec une probabilité 1. On sait également que la probabilité que l'image soit un chat est 0. Donc, le vecteur de gradient est $[(1 - 0.46), (0 - 0.54)] = [0.54, -0.46]$. Autrement dit, **on veut que la fonction apprenne à, étant donné cet input, augmenter la probabilité de chien et diminuer la probabilité de chat**, proportionnellement à l'erreur observée. La valeur de *loss* est la norme euclidienne de ce vecteur de gradient.

### Exercice
Premièrement, préparons les données en suivant les étapes suivantes:
1. Mettre tous les pixels de tous les canaux en un grand vecteur. On aura donc une matrice de taille (n_image, 3072)
2. Calculer la valeur moyenne de chaque pixel **sur les données d'entraînement uniquement** et centrer les données. En général, on a des meilleurs résultats sur des distributions sans biais.
3. Ajouter un élément à la fin de chaque ligne des $x$ pour ne pas avoir à calculer séparemment le biais $b$
4. Extraire une petite batch x_dev pour nous aider au développement.

In [ ]:
x_train = cifar10.x_train
x_test = cifar10.x_test
y_train = cifar10.y_train
y_test = cifar10.y_test

# Preprocessing: reshape the image data into rows
x_train = np.reshape(x_train, (x_train.shape[0], -1))
x_test = np.reshape(x_test, (x_test.shape[0], -1))


# Normalize the data: subtract the mean image
mean_image = np.mean(x_train, axis=0)
x_train -= mean_image
x_test -= mean_image

# add bias dimension and transform into columns
x_train = np.hstack([x_train, np.ones((x_train.shape[0], 1))])
x_test = np.hstack([x_test, np.ones((x_test.shape[0], 1))])

mask = np.random.choice(x_train.shape[0], 500, replace=False)
x_dev = x_train[mask]
y_dev = y_train[mask]

print(x_train.shape, x_test.shape, x_dev.shape)

Dans le fichier `dojo/softmax.py`, complétez la fonction `softmax_loss_naive`, puis roulez le code suivant.

In [ ]:
from dojo.softmax import softmax_loss_naive
import time

# Generate a random softmax weight matrix and use it to compute the loss.
W = np.random.randn(3073, 10) * 0.0001
loss, grad = softmax_loss_naive(W, x_dev, y_dev, 0.0)

# As a rough sanity check, our loss should be something close to -log(0.1).
print('loss: %f' % loss)
print('sanity check: %f' % (-np.log(0.1)))

Est-ce que votre fonction passe le sanity check? **Pourquoi s'attend-t-on à une valeur de $-\log(0.1)$**?

Maintenant, vérifiez que vous retournez le bon gradient en roulant le code suivant:

In [ ]:
# Complete the implementation of softmax_loss_naive and implement a (naive)
# version of the gradient that uses nested loops.
loss, grad = softmax_loss_naive(W, x_dev, y_dev, 0.0)

# As we did for the SVM, use numeric gradient checking as a debugging tool.
# The numeric gradient should be close to the analytic gradient.
from dojo.gradient_check import grad_check_sparse
f = lambda w: softmax_loss_naive(w, x_dev, y_dev, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

# similar to SVM case, do another gradient check with regularization
loss, grad = softmax_loss_naive(W, x_dev, y_dev, 1e2)
f = lambda w: softmax_loss_naive(w, x_dev, y_dev, 1e2)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

`grad_check_sparse` compare votre valeur de gradient avec un estimé numérique (versus un calcul analytique) calculé par une approche itérative (voir [wiki](https://en.wikipedia.org/wiki/Numerical_differentiation) pour plus de détails). Ce qu'il faut retenir, c'est que l'approximation numérique est trop lente à calculer pour les algos de machine learning mais peuvent nous permettre de vérifier notre travail.

Si `relative error` est un petit nombre, c'est gagné.


Maintenant, séparer votre ensemble d'entraînement en 2 partie: une parte de 49000 images pour l'entraînement (`x_train`) et une partie de 1000 images pour la validation (`x_val`). Itérez sur 

In [ ]:
x_train = x_train[1000:]
y_train = y_train[1000:]

x_val = x_train[:1000]
y_val = y_train[:1000]
print(x_train.shape, x_val.shape)

In [ ]:
# Use the validation set to tune hyperparameters (learning rate). 
# You should experiment with different ranges for the learning rates.
from dojo.linear_classifier import Softmax

results = {}
best_val = -1
best_softmax = None
learning_rates = [1e-7, 1e-4] # [min, max]

# Change num to change the number of learning_rates to try
lrs = np.log10(np.logspace(learning_rates[0], learning_rates[1], num=5, dtype='Float64'))  
num_iter = 2000


for lr in lrs:
    smc = Softmax()
    smc.train(x_train, y_train, learning_rate=lr, num_iters=num_iter)
    
    y_train_pred = smc.predict(x_train)
    y_val_pred = smc.predict(x_val)
    train_accuracy = np.mean(y_train == y_train_pred)
    validation_accuracy = np.mean(y_val == y_val_pred)
    
    results[lr] = (train_accuracy, validation_accuracy)
    if validation_accuracy > best_val:
        best_val = validation_accuracy
        best_softmax = smc

# Print out results.
for lr in sorted(results):
    train_accuracy, val_accuracy = results[lr]
    print('lr %e train accuracy: %f val accuracy: %f' % (lr, train_accuracy, val_accuracy))
    
print('best validation accuracy achieved during cross-validation: %f' % best_val)

In [ ]:
# evaluate on test set
# Evaluate the best softmax on test set
y_test_pred = best_softmax.predict(x_test)
test_accuracy = np.mean(y_test == y_test_pred)
print('softmax on raw pixels final test set accuracy: %f' % (test_accuracy, ))

#### Visualisation

La cellule suivante visualise les poids obtenus:

In [ ]:
# Visualize the learned weights for each class
w = best_softmax.W[:-1,:] # strip out the bias
w = w.reshape(32, 32, 3, 10)

w_min, w_max = np.min(w), np.max(w)
print(w_min, w_max)

classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for i in range(10):
    plt.subplot(2, 5, i + 1)

    # Rescale the weights to be between 0 and 255
    wimg = 255.0 * (w[:, :, :, i].squeeze() - w_min) / (w_max - w_min)
    plt.imshow(wimg.astype('uint8'))
    plt.axis('off')
    plt.title(classes[i])

## Partie 4 - Réseaux de neurones

Si vous avez bien compris la partie précédente, vous comprenez déjà les réseaux de neurones. Un réseaux de neurones est composé de:

1. **Une couche d'input**. Comme pour la classification linéaire, c'est l'image sous forme de vecteur.
2. **Couche(s) caché(s)**. Chaque neurone dans les couche cachées sont des classificateur linéaires, suivis de non-linéarités (plus de détails plus loin).
3. **Couche de sortie**. Comme pour le classificateur linéaire, il y a une sortie par classe, et on utilise la fonction softmax pour calculer le *loss* et le gradient.

![nn](http://cs231n.github.io/assets/nn1/neural_net2.jpeg)
![nn-zoom](http://cs231n.github.io/assets/nn1/neuron_model.jpeg)

L'idée générale d'un réseau de neurone est de combiner plusieur classificateurs linéaires pour avoir un pouvoir de représentation plus grand.

![lin-nn](http://image.slidesharecdn.com/actionrecognition-12661691655651-phpapp01/95/action-recognition-thesis-presentation-29-728.jpg?cb=1266147980)

On ajoute également une non linéarité (la fonction $f$ sur la deuxième image), ce qui augmente davantage le pouvoir de représentation du réseau. Trois fonctions d'activations sont populaires: tanh, sigmoid et ReLU (ReLU étant la plus fréquemment utilisée de nos jours).

![non-lin](https://imiloainf.files.wordpress.com/2013/11/activation_funcs1.png)

### Exercice

Utilisez Tensorflow pour battre le précision du classificateur linéaire de la partie précédente. Utilisez les `placeholder` tensorflow puisque c'est la manière la plus simple de procéder pour des petits datasets. Quelques pistes d'explorations:

- Jouez avec le nombre de couches et de neurones par couche
- Modifiez la l'initialisation des poids W et des biais b
- Changez de fonction d'activation (non-linéarité)

Quelques liens:
- https://www.tensorflow.org/how_tos/reading_data/#preloaded_data
- https://www.tensorflow.org/how_tos/threading_and_queues/
- http://playground.tensorflow.org/
- https://www.tensorflow.org/tutorials/mnist/tf/
- https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/tutorials/mnist/mnist.py

#### Sujets avancés

Certains sujets avancés n'ont pas été abordés durant ce dojo pour que la matière reste accessible et entre dans une seule journée. Si vous avez complétés votre réseau avec Tensorflow, vous pouvez explorer ces sujets:

- Réseaux de neurones convolutionnels (CNN)
- Dropout
- Batch-Normalization

Bien sûr, n'hésitez pas à demander aux animateurs de l'aide personnalisée et des explications sur ces sujets.